<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#导入相关的包" data-toc-modified-id="导入相关的包-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>导入相关的包</a></span></li><li><span><a href="#读取相关数据" data-toc-modified-id="读取相关数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>读取相关数据</a></span></li><li><span><a href="#对摘要信息进行分词" data-toc-modified-id="对摘要信息进行分词-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>对摘要信息进行分词</a></span></li><li><span><a href="#开始构建网络" data-toc-modified-id="开始构建网络-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>开始构建网络</a></span></li></ul></div>

# 导入相关的包

In [8]:
import pandas as pd
import jieba
import gensim
import warnings
import os
import re
from keras.layers.core import Activation, Dense,Dropout
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import nltk  #用来分词
import collections  #用来统计词频
import numpy as np

In [9]:
warnings.filterwarnings("ignore")
os.chdir("E:\\ccf\\Entity_Extraction")

# 读取相关数据

In [10]:
%%time
data_list = []
with open("raw_data\\DATA_1018\\DATA_1018\\DATA_T_RESULT.txt", encoding="utf-8") as rf:
    lines = rf.readlines()
    for line in lines:
        data_list.append(eval(line))
data_pd = pd.DataFrame()
for dl in data_list:
    pd_dl = pd.DataFrame(dl.get("entities"))
    data_pd = pd.concat([data_pd,pd_dl])
user_dict = pd.read_excel("raw_data\\DATA_1018\\DATA_1018\\NEGATIVE_WORDS.xls",encoding='utf-8')

Wall time: 3.78 s


In [13]:
data_pd["label"] = data_pd["eventLevel"].apply(lambda x: 1 if x == "正向" else (0 if x == "中性" else -1))
data_pd.head()
user_dict.head()
data_pd.to_hdf("raw_data\\entity-hdfs",key="data")

,digest,eventLevel,keywords,name,label
0,2017年第二季度，全省共完成并公布 监督 抽检 食品样品3061批次（其中 食用农产品...,中性,,山西省食品药品监督管理局,0
0,7月14日和17日，青岛市食品药品监督管理局官网公示2017年流通环节第三期食品安全监督...,负向,,青岛崂山大润发商业有限公司,-1
1,7月14日和17日，青岛市食品药品监督管理局官网公示2017年流通环节第三期食品安全监督抽检...,中性,,青岛市食品药品监督管理局,0
2,7月14日和17日，青岛市食品药品监督管理局官网公示2017年流通环节第三期食品安全监督抽检...,负向,,青岛金鼎广场有限公司,-1
0,施乐辉医用产品国际贸易有限公司召回髋关节置换系统、捷迈医疗国际贸易有限公司召回金属髓内钉手术...,负向,,强生医疗器械有限公司,-1


,序号,关键字,正向负向面
0,1,经济审判,负向
1,2,民主,正向
2,3,合同纠纷,负向
3,4,撤回起诉,负向
4,5,被投诉,负向


ImportError: HDFStore requires PyTables, "DLL load failed: 找不到指定的程序。" problem importing

# 对摘要信息进行分词

首先导入自定义字典，对摘要进行分词，并且只取其中包含中文的词

In [5]:
%%time
contents = data_pd.digest.values
contents_list = []
maxlen = 0
word_freqs = collections.Counter()  #词频
num_recs = 0 # 样本数
jieba.load_userdict("feature_data\\user_dict.txt")
for c in contents:
#     获取句子的最大长度
    if len(c) > maxlen:
        maxlen = len(c)
    cut_list = jieba.lcut(c)
    fenci_list_without_biaodian = []
    for fl in cut_list:
        pattern = re.compile(u"[\u4e00-\u9fa5]+")
        result = re.findall(pattern,fl)
        if  result != "":
            fenci_list_without_biaodian.extend(result)
#     分完词之后进行词频统计
    for word in fenci_list_without_biaodian:
        word_freqs[word] += 1
    num_recs += 1
    contents_list.append(fenci_list_without_biaodian)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\adc\AppData\Local\Temp\jieba.cache
Loading model cost 0.943 seconds.
Prefix dict has been built succesfully.


Wall time: 6.18 s


In [6]:
num_recs
maxlen
len(word_freqs)

14170

946

17794

总共有样本数14170个，总共的词有17794个，因此建立字典大小为15000，样本的最大长度为946
    1. 并且对于不在词汇表里的单词，把它们用伪单词 UNK 代替。 根据句子的最大长度 (max_lens)，我们可以统一句子的长度，把短句用 0 填充。
    2. 依前所述，我们把 VOCABULARY_SIZE 设为 15002。包含训练数据中按词频从大到小排序后的前 15000 个单词，外加一个伪单词 UNK 和填充单词 0。 最大句子长度 MAX_SENTENCE_LENGTH 设为800。 
    

In [7]:
max_features = 15000
max_sentence_length = 800

<font color=#0099ff size=6 face="黑体">接下来建立两个 lookup tables，分别是 word2index 和 index2word，用于单词和数字转换。 </font>

In [8]:
%%time
vocab_size = min(max_features, len(word_freqs)) + 2
word2index = {x[0]: i+2 for i, x in enumerate(word_freqs.most_common(max_features))}
word2index["PAD"] = 0
word2index["UNK"] = 1
index2word = {v:k for k, v in word2index.items()}

Wall time: 31 ms


<font size="5">下面就是根据 lookup table 把句子转换成数字序列了，并把长度统一到 MAX_SENTENCE_LENGTH， 不够的填 0 ， 多出的截掉。</font>

In [9]:
%%time
X = np.empty(num_recs,dtype=list)
label = []
labels = data_pd.label.values
print(labels)
print(len(labels))
for i in range(len(contents)):
    cut_list = jieba.lcut(contents[i])
    fenci_list_without_biaodian = []
    for fl in cut_list:
        pattern = re.compile(u"[\u4e00-\u9fa5]+")
        result = re.findall(pattern,fl)
        if  result != "":
            fenci_list_without_biaodian.extend(result)
    seqs = []
    for w in fenci_list_without_biaodian:
        if w in word2index:
            seqs.append(word2index[w])
        else:
            seqs.append(word2index["UNK"])
    X[i] = seqs
    label.append(labels[i])
X = sequence.pad_sequences(X, maxlen=max_sentence_length)
print(len(label))
print(label[0:5])
label = np.array(label)
print(label.shape)
x_train, x_test, y_train, y_test = train_test_split(X, label, test_size=0.2, random_state=42)

[ 0 -1  0 ...,  1  1  0]
14170
14170
[0, -1, 0, -1, -1]
(14170,)
Wall time: 5.4 s


In [10]:
x_train.shape
y_train.shape
x_test.shape
y_test.shape

(11336, 800)

(11336,)

(2834, 800)

(2834,)

In [11]:
y_train = np_utils.to_categorical(y_train,3)
y_test = np_utils.to_categorical(y_test,3)

In [12]:
y_train

array([[ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       ..., 
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.]])

<font size = 5>接下来构建深度网络的框架，这里损失函数用 binary_crossentropy， 优化方法用 adam。 <br><br/>至于 embedding_size , hidden_layer_size , 以及训练时用到的batch_size 和 num_epoch 这些超参数。<br><br/> 首先定义神经网络的超参数</font>

In [ ]:
embedding_size = 2048
hidden_layer_size = 1024
batch_size = 32
num_epoch= 10

# 开始构建网络

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size,embedding_size,input_length=max_sentence_length)) 
print(model.output_shape)
model.add(LSTM(hidden_layer_size, dropout=0.2, recurrent_dropout=0.2)) 
# print(model.output_shape)
# model.add(Dense(512,activation="relu"))
# model.add(Dropout(0.5))
print(model.output_shape)
# model.add(Dense(256,activation="relu"))
# model.add(Dropout(0.5))
print(model.output_shape)
model.add(Dense(64,activation="relu"))
print(model.output_shape) 
model.add(Dropout(0.5))
print(pd.DataFrame(model.get_weights()).shape)
model.add(Dense(3,activation='softmax')) 
print(model.summary())
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])
# 开始训练
model.fit(x_train, y_train, batch_size=batch_size, epochs=num_recs,validation_data=(x_test, y_test),shuffle=True)

(None, 800, 2048)
(None, 1024)
(None, 1024)
(None, 64)
(6, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 800, 2048)         30724096  
_________________________________________________________________
lstm_1 (LSTM)                (None, 1024)              12587008  
_________________________________________________________________
dense_1 (Dense)              (None, 64)                65600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 43,376,899
Trainable params: 43,376,899
Non-trainable params: 0
_________________________________________________________________
None
Train on 11336 samples, validate on 2834 samples
Epoch 1/14170
